**COLAB DEMO**



In [1]:
!git clone https://github.com/CompVis/latent-diffusion.git


.


In [2]:
!pip install -q torch transformers datasets evaluate accelerate nltk rouge_score
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q -e .
!pip install -q git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -q git+https://github.com/openai/CLIP.git@main#egg=clip

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [5]:
!ls -la /content/LAKE-RED


total 124
drwxr-xr-x 7 root root  4096 Apr 29 13:48 .
drwxr-xr-x 1 root root  4096 Apr 29 13:48 ..
drwxr-xr-x 2 root root  4096 Apr 29 13:48 assets
-rw-r--r-- 1 root root  1604 Apr 29 13:48 combine.py
drwxr-xr-x 3 root root  4096 Apr 29 13:48 demo
-rw-r--r-- 1 root root   212 Apr 29 13:48 demo.sh
-rw-r--r-- 1 root root   785 Apr 29 13:48 eval.sh
drwxr-xr-x 8 root root  4096 Apr 29 13:48 .git
-rw-r--r-- 1 root root  6954 Apr 29 13:48 inference_one_sample.py
drwxr-xr-x 8 root root  4096 Apr 29 13:48 ldm
-rw-r--r-- 1 root root 11357 Apr 29 13:48 LICENSE
-rw-r--r-- 1 root root 29280 Apr 29 13:48 main.py
-rw-r--r-- 1 root root  7292 Apr 29 13:48 README.md
-rw-r--r-- 1 root root  1093 Apr 29 13:48 split_validation_subset.py
-rw-r--r-- 1 root root  8350 Apr 29 13:48 test.py
-rw-r--r-- 1 root root   259 Apr 29 13:48 test.sh
-rw-r--r-- 1 root root   266 Apr 29 13:48 train.sh
drwxr-xr-x 2 root root  4096 Apr 29 13:48 user_study


In [3]:
import os
os.chdir('LAKE-RED')
!ls  # List files to verify we're in the right directory

'=1.0.8'      eval.sh			  README.md
'=2.0.0'      inference_one_sample.py	  setup.py
 assets       LAKE-RED			  split_validation_subset.py
 ckpt	      latent-diffusion		  taming-transformers
 combine.py   latent_diffusion.egg-info   test.py
 demo	      ldm			  test.sh
 demo_res     LICENSE			  train.sh
 demo.sh      main.py			  user_study


In [4]:
# Install PyTorch Lightning and other dependencies based on the imports
!pip install omegaconf pytorch-lightning tensorboard packaging pillow
!pip install torch-fidelity  # For evaluation later

# Make sure we have the right versions
!pip install torch torchvision

In [5]:


# Download pretrained autoencoding model
!wget -O /content/LAKE-RED/ldm/models/first_stage_models/vq-f4-noattn/model.ckpt "https://heibox.uni-heidelberg.de/f/9c6681f64bb94338a069/?dl=1"

# Download pretrained LDM
!wget -O /content/LAKE-RED/ldm/models/ldm/inpainting_big/last.ckpt "https://heibox.uni-heidelberg.de/f/4d9ac7ea40c64582b7c9/?dl=1"

# Download LAKE-RED model
!gdown "https://drive.google.com/uc?id=18SsVydpBPVwYNS5G_D42Ns0TlHJRzM-D" -O /content/LAKE-RED/ckpt/LAKERED.ckpt

--2025-04-29 15:17:40--  https://heibox.uni-heidelberg.de/f/9c6681f64bb94338a069/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/b729a5bf-1cc2-4036-bdb9-b02a776c9d3c/vq-f4-noattn.zip [following]
--2025-04-29 15:17:41--  https://heibox.uni-heidelberg.de/seafhttp/files/b729a5bf-1cc2-4036-bdb9-b02a776c9d3c/vq-f4-noattn.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 673997721 (643M) [application/zip]
Saving to: ‘/content/LAKE-RED/ldm/models/first_stage_models/vq-f4-noattn/model.ckpt’

/content/LAKE-RED/l 100%[===================>] 642.77M  12.3MB/s    in 55s     

2025-04-29 15:18:36 (11.7 MB/s) - ‘/content/LAKE-RED/ldm/models/first_stage_models/vq-f4-noattn/model.ckpt’ saved [6

In [6]:
# Download the LAKE-RED dataset
!gdown "https://drive.google.com/uc?id=1M96ipKc3nVK_Q_LViM6t9UjT8sayRxuZ" -O /conent/LAKE-RED/lakered_dataset.zip
!unzip -q lakered_dataset.zip

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1M96ipKc3nVK_Q_LViM6t9UjT8sayRxuZ

but Gdown can't. Please check connections and permissions.
unzip:  cannot find or open lakered_dataset.zip, lakered_dataset.zip.zip or lakered_dataset.zip.ZIP.


In [11]:
!cat /content/LAKE-RED/demo.sh

python inference_one_sample.py --image /content/LAKE-RED/demo/src/COD_CAMO_camourflage_00012.jpg \
                               --mask /content/LAKE-RED/demo/src/COD_CAMO_camourflage_00012.png \
                               --log_path /content/LAKE-RED/demo_res 

In [7]:
!git clone https://github.com/CompVis/latent-diffusion.git
!git clone https://github.com/CompVis/taming-transformers
!pip install -e ./taming-transformers
!pip install ipywidgets omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops

import sys
sys.path.append(".")
sys.path.append('./taming-transformers')

from torch import inf
!pip install taming-transformers
!pip install taming-transformers-rom1504


from taming.models import vqgan # checking correct import from taming

fatal: destination path 'latent-diffusion' already exists and is not an empty directory.
fatal: destination path 'taming-transformers' already exists and is not an empty directory.
Obtaining file:///content/LAKE-RED/taming-transformers
  Preparing metadata (setup.py) ... done
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Uninstalling taming-transformers-0.0.1:
      Successfully uninstalled taming-transformers-0.0.1
  Running setup.py develop for taming-transformers


In [8]:
# Downgrade PyTorch Lightning to version 1.9.0
!pip install pytorch-lightning==1.9.0
# Install additional dependencies
!pip install albumentations omegaconf


## running **demo**

In [39]:
%cd /content/LAKE-RED
!sh demo.sh

/content
CUDA initialized
Current device: 0
Device name: Tesla T4

GPU Memory after forced allocation:
Allocated: 11.44482421875 MB
Cached: 20.0 MB
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Called with args:
Namespace(yaml_path='ldm/models/ldm/inpainting_big/config_LAKERED.yaml', model_path='ckpt/LAKERED.ckpt', log_path='/content/LAKE-RED/demo_res', image='/content/LAKE-RED/demo/src/COD_CAMO_camourflage_00012.jpg', mask='/content/LAKE-RED/demo/src/COD_CAMO_camourflage_00012.png', batchsize=9, isReplace=False, dilate_kernel=2, Steps=50)
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 387.25 M params.
Keeping EMAs of 433.
making attention of type 'none' with 512 in_channels
Working with z of 

running demo.sh directly


In [46]:
%cd /content/LAKE-RED
!python inference_one_sample.py --image demo/src/COD_CAMO_camourflage_00018.jpg \
                               --mask demo/src/COD_CAMO_camourflage_00018.png \
                               --log_path demo_res

/content
CUDA initialized
Current device: 0
Device name: Tesla T4

GPU Memory after forced allocation:
Allocated: 11.44482421875 MB
Cached: 20.0 MB
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Called with args:
Namespace(yaml_path='ldm/models/ldm/inpainting_big/config_LAKERED.yaml', model_path='ckpt/LAKERED.ckpt', log_path='demo_res', image='demo/src/COD_CAMO_camourflage_00018.jpg', mask='demo/src/COD_CAMO_camourflage_00018.png', batchsize=9, isReplace=False, dilate_kernel=2, Steps=50)
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 387.25 M params.
Keeping EMAs of 433.
making attention of type 'none' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attent

checking for gpu, image loading (not required to run)

In [17]:
!nvidia-smi

Tue Apr 29 15:48:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   74C    P0             35W /   70W |       0MiB /  15360MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
import cv2
img = cv2.imread('demo/src/COD_CAMO_camourflage_00012.jpg')
print(img is not None)  # Should print True if image was read successfully

True


In [19]:
from PIL import Image
img = Image.open('demo/src/COD_CAMO_camourflage_00012.jpg')
print(img.size)  # Should print the image dimensions

(632, 420)


In [25]:
import torch
print("GPU Memory allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("GPU Memory cached:", torch.cuda.memory_reserved() / 1024**2, "MB")

GPU Memory allocated: 0.0 MB
GPU Memory cached: 0.0 MB


In [32]:
!ls -lh ckpt/LAKERED.ckpt

-rw-r--r-- 1 root root 6.0G Jun 10  2024 ckpt/LAKERED.ckpt
